# Projet Olist : Analyse Data-Driven

## 1. Comprendre les données

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import des tables
import pandas as pd
import numpy as np
customers=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_customers_dataset.csv')
geolocation=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_geolocation_dataset.csv')
order_items=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_order_items_dataset.csv')
order_payments=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_order_payments_dataset.csv')
order_reviews=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_order_reviews_dataset.csv')
orders=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_orders_dataset.csv')
products=pd.read_csv('/content/drive/MyDrive/archive (8)/products.csv')
sellers=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_sellers_dataset.csv')
reviewtranslted=pd.read_csv('/content/drive/MyDrive/archive (8)/olist_order_reviews_dataset_translated.csv')


In [ ]:
orders.head()

In [ ]:
orders.info()

In [ ]:
orders.isnull().sum()

In [ ]:
orders['order_status'].unique()

## 2. Nettoyer les données

In [ ]:
# Changement type culomn
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp']).dt.normalize()
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date']).dt.normalize()
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date']).dt.normalize()
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date']).dt.normalize()
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at']).dt.normalize()

## Vérification des commandes et détails produits

In [ ]:
orders_not_in_items = orders[~orders['order_id'].isin(order_items['order_id'])]

In [ ]:
orders_not_in_items.shape[0]

In [ ]:
items_not_in_orders = order_items[~order_items['order_id'].isin(orders['order_id'])]

In [ ]:
orders_value=order_payments.groupby('order_id')['payment_value'].sum()

In [ ]:
# Chaque commande avec total payement
orders_with_payment=orders.merge(order_payments,on='order_id',how='left')
# filtre sur statut de commande
filtered_orders_with_payment = orders_with_payment[~orders_with_payment['order_status'].isin(['canceled', 'unavailable'])]


In [ ]:
#Calcul de différence entre les Dates:
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Calculate differences starting from order_purchase_timestamp
orders['days_to_approve'] = (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.days
orders['days_to_carrier'] = (orders['order_delivered_carrier_date'] - orders['order_purchase_timestamp']).dt.days
orders['days_to_customer_delivery'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['days_to_estimated_delivery'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days

# Calculate differences starting from order_approved_at
orders['days_approved_to_carrier'] = (orders['order_delivered_carrier_date'] - orders['order_approved_at']).dt.days
orders['days_approved_to_customer_delivery'] = (orders['order_delivered_customer_date'] - orders['order_approved_at']).dt.days
orders['days_approved_to_estimated_delivery'] = (orders['order_estimated_delivery_date'] - orders['order_approved_at']).dt.days

# Calculate differences starting from order_delivered_carrier_date
orders['days_carrier_to_customer_delivery'] = (orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']).dt.days
orders['days_carrier_to_estimated_delivery'] = (orders['order_estimated_delivery_date'] - orders['order_delivered_carrier_date']).dt.days

# Calculate differences starting from order_delivered_customer_date
orders['days_customer_delivery_to_estimated_delivery'] = (orders['order_estimated_delivery_date'] - orders['order_delivered_customer_date']).dt.days

display(orders.head())

In [ ]:
#Correction date
orders_to_swap = ['a1abeb653a4d4cd1e142ccb8c82cd069',
 '383aa8b2724fe452d9ccd9934a8c628b',
 'cb1134f9010d242e9515ad1c78ec0c39',
 'dceb62e8fa94b46006c9554fed743df0',
 '5f9d46795c3126674e52becb3a1a517f',
 '8c78d01de3a9009e23d6877a7cc9be20',
 'b27af682321527a6349f1761eb3f360c',
 '1cc3ae63caffff2d6c3ee3e78e074acf',
 'e37f11cae9985ca58f0b56f268720537',
 'fa3e37584f4fdb1ded0e0de700dfcb4e',
 'c1e2bf2b7dd3309f2f5356c6b63968fa',
 'b866af202be0692766081310cd4085e1',
 '29941903985f944b0ffc49c479c1547d',
 '76458889992169d3135b264dc13aec67',
 '0922ee1619de7b995648e5a8407afb91',
 'fbffbaaa30f4077e87ec6acdb835b278',
 '3ee6694bb99d5a6db0deceffcd099597',
 '19feb5627c41ea1b36a8e50a469b3644',
 'd5558a097766363b8e76b38c43332e8a',
 '771c4f1f521f462e4b95619e648aaeab',
 'ea1dcb4757a844d2642547797bd5feb0',
 '99ad48402644a3968f2481defdc57947',
 'f688669f48063536e082bb32d634cd46']
mask = orders['order_id'].isin(orders_to_swap)
orders.loc[mask, ['order_delivered_carrier_date', 'order_delivered_customer_date']] = \
orders.loc[mask, ['order_delivered_customer_date', 'order_delivered_carrier_date']].values

In [ ]:
# update_delivery_date
orders_to_update_delivery_date = [
    '2d1e2d5bf4dc7227b3bfebb81328c15f',
    'f5dd62b788049ad9fc0526e3ad11a097',
    '2ebdfc4f15f23b91474edf87475f108e',
    'e69f75a717d64fc5ecdfae42b2e8e086',
    '0d3268bad9b086af767785e3f0fc0133',
    '2d858f451373b04fb5c984a1cc2defaf',
    'ab7c89dc1bf4a1ead9d6ec1ec8968a84',
    '20edc82cf5400ce95e1afacc25798b31'
]

mask = orders['order_id'].isin(orders_to_update_delivery_date)
orders.loc[mask, 'order_delivered_customer_date'] = orders.loc[mask, 'order_estimated_delivery_date']

display(orders[mask][['order_id', 'order_estimated_delivery_date', 'order_delivered_customer_date']])

In [ ]:
# Suppression lignes inutiles
orders_to_drop = ['7c48bb55e8e4f7e56d412e9653db37bc', '4021cd7611d6d9ce5ffcd24817fc374f']
orders = orders[~orders['order_id'].isin(orders_to_drop)]

## 3. Joindre les données

In [ ]:
# Exemple de jointure orders avec order_items
orders_items_merged = pd.merge(orders, order_items, on='order_id', how='left')
orders_items_merged.head()

In [ ]:
#Export du fichier format CSV
orders_items_merged.to_csv('orders_items_merged.csv', index=False)

## Inventory turnover rate

In [ ]:
orderitems_product=order_items.merge(products,how='left',on='product_id')

In [ ]:
inventory_turnover = (
    orderitems_product.groupby('product_category_name')
      .agg(
          distinct_orders=('order_id', pd.Series.nunique),
          distinct_products=('product_id', pd.Series.nunique)
      )
      .assign(inventory_turnover_rate=lambda x: x['distinct_orders'] / x['distinct_products'])
      .reset_index()
      .sort_values(by='inventory_turnover_rate', ascending=False)
)

print(inventory_turnover)

## satisfaction_percentages

In [ ]:
order_reviews['satisfaction_status'] = np.where(order_reviews['review_score'] >= 4, 'satisfied', 'unsatisfied')

satisfaction_percentages = order_reviews['satisfaction_status'].value_counts(normalize=True) * 100

print('Percentage of satisfied vs. unsatisfied customers:')
print(satisfaction_percentages)

In [ ]:
# Ajout total payement pour une commande dans table Orders
order_id_to_update = 'bfbd0f9bdef84302105ad712db648a6c'
new_payment_value = 143.46

orders_pymt_global.loc[orders_pymt_global['order_id'] == order_id_to_update, 'payment_value'] = new_payment_value

display(orders_pymt_global[orders_pymt_global['order_id'] == order_id_to_update])

In [ ]:
# Compter le nombre de commandes par ville et statut
city_status_counts = orders_noitems.groupby(['customer_state', 'order_status']).size().reset_index(name='count')

print(city_status_counts.head())

## 4. Visualiser les données

In [ ]:
#visulation chiffre d'affaire
import matplotlib.pyplot as plt
import seaborn as sns

# Group by order_purchase_timestamp and sum payment_value
daily_payments = orders.groupby('order_purchase_timestamp')['payment_value'].sum().reset_index()

# Create the line plot
plt.figure(figsize=(15, 7))
sns.lineplot(x='order_purchase_timestamp', y='payment_value', data=daily_payments)

plt.title('Evolution of Payment Value over Purchase Date')
plt.xlabel('Purchase Date')
plt.ylabel('Total Payment Value')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#commandes livrés intime vs Late par seller
# Sellers = axe X
sellers = late_more3['seller_name']

# Valeurs
intime_cmdes = late_more3['nb_in_time']
late_cmdes = late_more3['nb_late']

# Positionnement
x = np.arange(len(sellers))
width = 0.4

plt.figure(figsize=(14,6))

plt.bar(x - width/2, intime_cmdes, width, label='commandes intime')
plt.bar(x + width/2, late_cmdes, width, label='Commandes late')

plt.xticks(x, sellers, rotation=90)
plt.ylabel("Nombre de commandes")
plt.title("Comparaison intime vs Late par seller (late > 3)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#Nombre de commandes validées le jour même
plt.figure(figsize=(12,6))
monthly_zero.plot(kind='bar')
plt.title("Nombre de commandes validées le jour même par année et mois")
plt.xlabel("Année-Mois")
plt.ylabel("Nombre de commandes")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



In [ ]:
# commandes validées dans 5 jours
import pandas as pd

# Définir les intervalles 
bins = [0, 5, All_info['days_toapprov'].max()]  
labels = ['0-5', '6+']  # noms des groupes

# Créer une nouvelle colonne 'days_group'
All_info['days_group'] = pd.cut(All_info['days_toapprov'], bins=bins, labels=labels, right=True, include_lowest=True)
zero_to_5 = All_info[All_info['days_group'] == '0-5']

# Extraire année et mois
zero_to_5['year_month'] = zero_to_5['order_purchase_timestamp'].dt.to_period('M')

# Grouper par année-mois
monthly_counts = zero_to_5.groupby('year_month').size()

# Tracer
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
monthly_counts.plot(kind='bar')
plt.title("Nombre de commandes validées en 0-5 jours par année et mois")
plt.xlabel("Année-Mois")
plt.ylabel("Nombre de commandes")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


## 5. Analyse ML / Prévision

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
monthly_orders = orders.groupby([
    orders['order_purchase_timestamp'].dt.year.rename('year'),
    orders['order_purchase_timestamp'].dt.month.rename('month')
])['order_id'].nunique().reset_index(name='order_count')
display(monthly_orders)

In [ ]:
monthly_orders["date"] = pd.to_datetime(
    monthly_orders["year"].astype(str) + "-" + monthly_orders["month"].astype(str) + "-01"
) + pd.offsets.MonthEnd(0)

ts = monthly_orders.set_index("date")["order_count"].sort_index()

display(ts.head())
display(ts.tail())

In [ ]:
from prophet import Prophet

# Préparation données pour Prophet
orders_ts = orders_clean.groupby('order_purchase_timestamp').size().reset_index(name='y')
orders_ts.rename(columns={'order_purchase_timestamp':'ds'}, inplace=True)

# Modèle Prophet
model = Prophet()
model.fit(orders_ts)

# Prévision
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

# Visualisation
fig = model.plot(forecast)
plt.title('Prévision des commandes avec Prophet')
plt.show()